In [ ]:
import nltk
import sklearn
import matplotlib as plt

In [ ]:
#Task 1: N-gram Generation

In [ ]:
#Task 2: N-Gram Modeling

In [ ]:
#Task 3: Handling unseen n-grams

In [ ]:
#Task 4: Interpolation